In [11]:
import pandas as pd
import os
import requests
import json
import csv
import io
from collections import defaultdict
from time import time
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
#from flair.data import Sentence
#from flair.nn import Classifier
from chembl_webresource_client.new_client import new_client

# 1. Get targets from disease

In [61]:
"""
insert code here that
1. Takes an EFO as input
2. displays a dataframe of targets for that disease
3. uploads that dataframe to a table in planetscale called "disease_to_target"

"""
from sqlalchemy import create_engine
from dotenv import dotenv_values
from sqlalchemy import text
config = dotenv_values('database_url.env')
url = config['DATABASE_URL']

engine = create_engine(url, echo=False)



## Set disease_id variable for desired disease

disease_id = "EFO_0005537"


with engine.begin() as conn:
    query = text("SELECT * FROM disease_to_target WHERE disease_id = '{disease_id}';".format(disease_id = disease_id))
    disease_df = pd.read_sql(query, conn)

display(disease_df)


target_ids = disease_df.sort_values(by = ['association_score'], ascending=False)['target_ensemble_id'].values
print(target_ids)
target_id = target_ids[1]

print(target_id)
print(type(target_id))

,level_0,index,disease_id,target_ensemble_id,target_chembl_id,association_score
0,0,0,EFO_0005537,ENSG00000184292,CHEMBL3856163,0.431490
1,1,1,EFO_0005537,ENSG00000198900,CHEMBL1781,0.424207
2,2,2,EFO_0005537,ENSG00000120217,CHEMBL3580522,0.405569
3,3,3,EFO_0005537,ENSG00000258947,CHEMBL2597,0.402827
4,4,4,EFO_0005537,ENSG00000137267,CHEMBL3797012,0.399973
...,...,...,...,...,...,...
5012,5012,5012,EFO_0005537,ENSG00000135164,CHEMBL3707469,0.001478
5013,5013,5013,EFO_0005537,ENSG00000115665,CHEMBL4507,0.001478
5014,5014,5014,EFO_0005537,ENSG00000146109,CHEMBL4507,0.001478
5015,5015,5015,EFO_0005537,ENSG00000225830,CHEMBL4507,0.001478


['ENSG00000184292' 'ENSG00000198900' 'ENSG00000120217' ...
 'ENSG00000072133' 'ENSG00000069020' 'ENSG00000183542']
ENSG00000198900
<class 'str'>


# 2. Get compounds and assays

In [56]:
"""
insert code here that
1. takes a target as input
2. searches the database for compounds and assays for that target
3. displays a dataframe with at least these columns: target id, compound id, assay id
"""
engine = create_engine(url, echo=False)
#target_id = 'ENSG00000120217'


with engine.begin() as conn:
    query = text("SELECT * FROM target_to_compounds WHERE target_ensemble_id='{target}';".format(target=target_id))
    target_to_compounds_df = pd.read_sql(query, conn)

display(target_to_compounds_df)





,level_0,index,compound_id,smiles,type,standard_value,assay_id,assay_type,assay_description,document_id,abstract,target_ensemble_id
0,12279,0,CHEMBL3968023,C[n+]1ccc2c3c(nc4cc5c(cc42)OCO5)-c2ccccc2-c31....,IC50,0.45,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid...",ENSG00000198900
1,12280,1,CHEMBL3914791,C[n+]1c2c3c(nc(-c4ccccc4)cc3c3ccccc31)-c1ccccc...,IC50,0.475,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid...",ENSG00000198900
2,12281,2,CHEMBL3896867,[Br-].c1ccc(C[n+]2ccc3c4c(nc5cc6c(cc53)OCO6)-c...,IC50,0.52,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid...",ENSG00000198900
3,12282,3,CHEMBL3889816,[Br-].c1ccc(C[n+]2ccc3c4c(nc5ccccc53)-c3ccccc3...,IC50,0.57,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid...",ENSG00000198900
4,12283,4,CHEMBL3984603,Cc1cc2c3c(nc4ccccc42)-c2ccccc2-c3[n+]1C.[I-],IC50,0.7,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid...",ENSG00000198900
...,...,...,...,...,...,...,...,...,...,...,...,...
451,12730,451,CHEMBL426092,O=C1Oc2ccc(O)cc2/C1=C\c1ccc(O)cc1,IC50,555000.0,CHEMBL827155,B,Inhibitory concentration against DNA topoisome...,CHEMBL1143530,Isoaurostatin A (IAS-A) isolated from Thermomo...,ENSG00000198900
452,12731,452,CHEMBL9164,CCCCCCC(=O)O[C@H](CC=C(C)C)C1=CC(=O)c2c(O)ccc(...,IC50,625000.0,CHEMBL663903,B,Inhibitory activity against DNA topoisomerase-...,CHEMBL1128571,Compounds bearing an acyl group of a various s...,ENSG00000198900
453,12732,453,CHEMBL372372,COc1cc(/C=C2/C(=O)Oc3ccccc32)ccc1O,IC50,630000.0,CHEMBL827155,B,Inhibitory concentration against DNA topoisome...,CHEMBL1143530,Isoaurostatin A (IAS-A) isolated from Thermomo...,ENSG00000198900
454,12733,454,CHEMBL381195,COc1ccc(/C=C2/C(=O)Oc3ccccc32)cc1O,IC50,800000.0,CHEMBL827155,B,Inhibitory concentration against DNA topoisome...,CHEMBL1143530,Isoaurostatin A (IAS-A) isolated from Thermomo...,ENSG00000198900


# 3. Propose a new compound

In [67]:
"""
insert code here that
1. trains a model to predict ic50 values for a compound on a given target
2. downloads ALL the compounds in the database (independent of target)
3. runs the model to predict ic50 values for each compound in the database
4. displays a dataframe that has the following two columns : compound, predicted ic50
5. prints out the compound that has the highest ic50 value that has NOT been tested on the target already.
"""
# 1. Train Model
import deepchem as dc
import numpy as np

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

#!git clone https://github.com/dsalinasduron-msmary/chemical_informatics.git

import os
import pickle
#os.listdir()
#os.chdir('chemical_informatics')
#os.getcwd()

f = open('ENSG00000128191dataframe.pickle','rb')
compound_dataset = pickle.load(f)
f.close()
display(compound_dataset)

smiles = compound_dataset['smiles']
IC50 = compound_dataset['standard value']
featurizer = dc.feat.ConvMolFeaturizer()
compound_dataset['featurized'] = featurizer.featurize(smiles)
compound_dataset['divided values'] = compound_dataset['standard value'].astype(float).div(108000)
compound_dataset['pIC50'] = np.log10(compound_dataset['divided values'].astype(float)).mul(-1)
compound_dataset['number'] = list(range(0,len(compound_dataset)))
display(compound_dataset)

#compound_dataset['pIC50'].hist()

x = compound_dataset
f = open('x.pickle','wb')
pickle.dump(x,f)
f.close()

import pandas as pd
training_dataset = compound_dataset.sample(frac = 0.7)

#training_dataset.featurized[0].n_feat

testing_dataset = (compound_dataset[~compound_dataset['number'].isin(training_dataset['number'])])
display(testing_dataset)

numpy_training_dataset = dc.data.NumpyDataset(X=training_dataset['featurized'],y=training_dataset['pIC50'].astype(float), ids=training_dataset['smiles'])
numpy_testing_dataset = dc.data.NumpyDataset(X=testing_dataset['featurized'],y=testing_dataset['pIC50'].astype(float), ids=testing_dataset['smiles'])
display(numpy_training_dataset)
display(numpy_testing_dataset)

model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2, dense_layer_size=10, model_dir="/content/chemical_informatics")

model.fit(numpy_training_dataset, nb_epoch=10)

model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2, dense_layer_size=10, model_dir="/content/chemical_informatics")
model.restore()

metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score:", model.evaluate(numpy_training_dataset, [metric]))
print("Test set score:", model.evaluate(numpy_testing_dataset, [metric]))


#2. Download all compounds in database
with engine.begin() as conn:
    query = text("SELECT compound_id,smiles FROM target_to_compounds;")
    compounds_df = pd.read_sql(query, conn)

predict_list = []
for smile in range(0, len(compounds_df)):
    predict_list.append(None)
print(len(predict_list))

compounds_df['predicted_pIC50'] = predict_list

curated_compounds_df = compounds_df.dropna(subset=['smiles'])

new_smiles = curated_compounds_df['smiles']
curated_compounds_df['featurized'] = featurizer.featurize(new_smiles)

#3. run the model to predict ic50 values for each compound in the database
dataset = dc.data.NumpyDataset(X=curated_compounds_df['featurized'], y=curated_compounds_df['predicted_pIC50'], ids=curated_compounds_df['smiles'])
curated_compounds_df['predicted_pIC50'] = model.predict(dataset)

# 4. displays a dataframe that has the following two columns : compound, predicted ic50
display(curated_compounds_df)

# 5. print out the compound that has the highest ic50 value that has NOT been tested on the target already.

curated_compounds_no_target = (curated_compounds_df[~curated_compounds_df['smiles'].isin(target_to_compounds_df['smiles'])])

print(len(curated_compounds_no_target) <= len(curated_compounds_df) - len(target_to_compounds_df))

df = curated_compounds_no_target.sort_values(by = ['predicted_pIC50'], ascending=False)
display(df)
compound_ids = df['compound_id'].values

#display(df)
#print(compound_ids)
compound_id = compound_ids[0]

print(compound_id)




2.13.0


,compound id,smiles,type,standard value,assay_id
0,CHEMBL2088421,COc1ccc(NC(=O)N(CCCCC2CCCCC2)CCc2ccc(SC(C)(C)C...,IC50,0.28,CHEMBL2089717
1,CHEMBL21241,CC(C)(Sc1ccc(CCN(CCCCC2CCCCC2)C(=O)NC2CCCCC2)c...,IC50,0.46,CHEMBL2089717
2,CHEMBL2088422,COc1cccc(NC(=O)N(CCCCC2CCCCC2)CCc2ccc(SC(C)(C)...,IC50,0.59,CHEMBL2089717
3,CHEMBL20392,COc1ccccc1NC(=O)N(CCCCC1CCCCC1)CCc1ccc(SC(C)(C...,IC50,1.93,CHEMBL2089717
4,CHEMBL424685,CC[C@@](C)(Cc1ccc(OCCCOc2ccc(C3CCCCC3)cc2Cl)cc...,IC50,2.0,CHEMBL867227
...,...,...,...,...,...
994,CHEMBL981,CC(C)(Oc1ccc(C(=O)c2ccc(Cl)cc2)cc1)C(=O)O,IC50,68000.0,CHEMBL824654
995,CHEMBL2153842,CC(C)(Oc1ccc(CCNC(=O)c2ccc(Cl)cc2)cc1)C(=O)NS(...,IC50,68500.0,CHEMBL2154210
996,CHEMBL4475797,CCOc1ccc(-c2ccc(CCCc3nn(CC)c(=O)n3-c3ccccc3F)c...,IC50,89300.0,CHEMBL4332153
997,CHEMBL981,CC(C)(Oc1ccc(C(=O)c2ccc(Cl)cc2)cc1)C(=O)O,IC50,106000.0,CHEMBL983194


,compound id,smiles,type,standard value,assay_id,featurized,divided values,pIC50,number
0,CHEMBL2088421,COc1ccc(NC(=O)N(CCCCC2CCCCC2)CCc2ccc(SC(C)(C)C...,IC50,0.28,CHEMBL2089717,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000003,5.586266,0
1,CHEMBL21241,CC(C)(Sc1ccc(CCN(CCCCC2CCCCC2)C(=O)NC2CCCCC2)c...,IC50,0.46,CHEMBL2089717,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000004,5.370666,1
2,CHEMBL2088422,COc1cccc(NC(=O)N(CCCCC2CCCCC2)CCc2ccc(SC(C)(C)...,IC50,0.59,CHEMBL2089717,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000005,5.262572,2
3,CHEMBL20392,COc1ccccc1NC(=O)N(CCCCC1CCCCC1)CCc1ccc(SC(C)(C...,IC50,1.93,CHEMBL2089717,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000018,4.747866,3
4,CHEMBL424685,CC[C@@](C)(Cc1ccc(OCCCOc2ccc(C3CCCCC3)cc2Cl)cc...,IC50,2.0,CHEMBL867227,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000019,4.732394,4
...,...,...,...,...,...,...,...,...,...
994,CHEMBL981,CC(C)(Oc1ccc(C(=O)c2ccc(Cl)cc2)cc1)C(=O)O,IC50,68000.0,CHEMBL824654,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.629630,0.200915,994
995,CHEMBL2153842,CC(C)(Oc1ccc(CCNC(=O)c2ccc(Cl)cc2)cc1)C(=O)NS(...,IC50,68500.0,CHEMBL2154210,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.634259,0.197733,995
996,CHEMBL4475797,CCOc1ccc(-c2ccc(CCCc3nn(CC)c(=O)n3-c3ccccc3F)c...,IC50,89300.0,CHEMBL4332153,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.826852,0.082572,996
997,CHEMBL981,CC(C)(Oc1ccc(C(=O)c2ccc(Cl)cc2)cc1)C(=O)O,IC50,106000.0,CHEMBL983194,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.981481,0.008118,997


,compound id,smiles,type,standard value,assay_id,featurized,divided values,pIC50,number
4,CHEMBL424685,CC[C@@](C)(Cc1ccc(OCCCOc2ccc(C3CCCCC3)cc2Cl)cc...,IC50,2.0,CHEMBL867227,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000019,4.732394,4
5,CHEMBL3264919,CCn1c(CCCc2ccc(OC(C)(C)C(=O)NS(=O)(=O)c3ccccc3...,IC50,2.7,CHEMBL3269597,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000025,4.602060,5
7,CHEMBL382178,CC[C@@](C)(Cc1ccc(OCCCOc2ccc(OC(F)(F)F)cc2Cl)c...,IC50,3.0,CHEMBL867227,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000028,4.556303,7
10,CHEMBL3264925,CCn1c(CCCc2ccc(OC(C)(C)C(=O)NS(=O)(=O)C3CCCC3)...,IC50,4.6,CHEMBL3269597,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000043,4.370666,10
12,CHEMBL234659,COc1cc(OCC(=O)O)c(C)cc1SCc1ccc(OCc2ccc(C(F)(F)...,IC50,4.7,CHEMBL896361,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.000044,4.361326,12
...,...,...,...,...,...,...,...,...,...
978,CHEMBL1808553,CC(C)C(Sc1nc2cc(Cl)ccc2s1)C(=O)NS(=O)(=O)c1ccccc1,IC50,27900.0,CHEMBL1809825,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.258333,0.587820,978
981,CHEMBL1173474,CCCCCCCCCCCCCCCCCCCCCC(=O)O,IC50,30000.0,CHEMBL3069175,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.277778,0.556303,981
985,CHEMBL1808551,CCCC(Sc1nc2ccc(OCC)cc2s1)C(=O)NS(=O)(=O)c1ccccc1,IC50,31000.0,CHEMBL1809825,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.287037,0.542062,985
994,CHEMBL981,CC(C)(Oc1ccc(C(=O)c2ccc(Cl)cc2)cc1)C(=O)O,IC50,68000.0,CHEMBL824654,<deepchem.feat.mol_graphs.ConvMol object at 0x...,0.629630,0.200915,994


<NumpyDataset X.shape: (699,), y.shape: (699,), w.shape: (699,), ids: ['CCCC(Sc1nc2cc(Cl)ccc2s1)C(=O)NS(C)(=O)=O'
 'O=C(O)c1ccc(COc2ccc3ccccc3c2/C=N/c2ccc(N3CCOCC3)cc2)cc1'
 'CCCn1c(CCCc2ccc(OC(C)(C)C(=O)O)cc2)nn(Cc2ccc(C)cc2)c1=O' ...
 'CCCc1c(OCCCSc2ccc(CC(=O)O)cc2Cl)ccc2c(C(F)(F)F)noc12'
 'CCCc1c(OCCCN(CC)c2ccc(CC(=O)O)cc2)ccc2c(C(F)(F)F)noc12'
 'Cc1oc(C2CCCCC2)nc1CCOc1ccc(C[C@](C)(Oc2ccccc2)C(=O)O)cc1'], task_names: [0]>

...

C:\Users\j.p.marinelli\AppData\Local\Temp\ipykernel_13944\2436685952.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curated_compounds_df['featurized'] = featurizer.featurize(new_smiles)
C:\Users\j.p.marinelli\AppData\Local\Temp\ipykernel_13944\2436685952.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curated_compounds_df['predicted_pIC50'] = model.predict(dataset)


,compound_id,smiles,predicted_pIC50,featurized
0,CHEMBL570841,N=C1NC(=O)/C(=C/c2ccc(O)cc2)S1,1.235434,<deepchem.feat.mol_graphs.ConvMol object at 0x...
1,CHEMBL570841,N=C1NC(=O)/C(=C/c2ccc(O)cc2)S1,1.235434,<deepchem.feat.mol_graphs.ConvMol object at 0x...
2,CHEMBL4128424,O=C(COc1ccc2c(-c3ccccc3)cc(=O)oc2c1)NN1C(=O)CS...,0.904736,<deepchem.feat.mol_graphs.ConvMol object at 0x...
3,CHEMBL107,COc1cc2c(c(OC)c1OC)-c1ccc(OC)c(=O)cc1[C@@H](NC...,0.562155,<deepchem.feat.mol_graphs.ConvMol object at 0x...
4,CHEMBL4127437,COc1cc(C2SCC(=O)N2NC(=O)COc2ccc3c(-c4ccccc4)cc...,0.860161,<deepchem.feat.mol_graphs.ConvMol object at 0x...
...,...,...,...,...
13747,CHEMBL7976,O=C(/C=C/c1ccccc1)c1ccccc1,0.663076,<deepchem.feat.mol_graphs.ConvMol object at 0x...
13748,CHEMBL107,COc1cc2c(c(OC)c1OC)-c1ccc(OC)c(=O)cc1[C@@H](NC...,0.562155,<deepchem.feat.mol_graphs.ConvMol object at 0x...
13749,CHEMBL18869,COc1cccc(-c2nc(O)c3cc(OC)ccc3n2)c1,0.386904,<deepchem.feat.mol_graphs.ConvMol object at 0x...
13750,CHEMBL278647,COc1cccc(-c2nc(O)c3cc(OC)c(OC)cc3n2)c1,0.566823,<deepchem.feat.mol_graphs.ConvMol object at 0x...


True


,compound_id,smiles,predicted_pIC50,featurized
2308,CHEMBL4471904,O=c1nc(SCc2cccc(O)c2)nc(SCc2ccc(Cl)cc2)[nH]1,2.211578,<deepchem.feat.mol_graphs.ConvMol object at 0x...
2346,CHEMBL4474662,NC(=O)c1cccc(CSc2nc(SCc3ccc(Cl)cc3)[nH]c(=O)n2)c1,2.204773,<deepchem.feat.mol_graphs.ConvMol object at 0x...
8733,CHEMBL2151921,O=C(Cc1nc(N2CCOCC2)cc(=O)[nH]1)Nc1cccc(Cl)c1,2.199797,<deepchem.feat.mol_graphs.ConvMol object at 0x...
8564,CHEMBL3696288,O=C(Cc1nc(N2CCOCC2)cc(=O)[nH]1)Nc1cccc(OC(F)(F...,2.197530,<deepchem.feat.mol_graphs.ConvMol object at 0x...
9853,CHEMBL2151920,O=C(Cc1nc(N2CCOCC2)cc(=O)[nH]1)Nc1ccc(Cl)cc1,2.196099,<deepchem.feat.mol_graphs.ConvMol object at 0x...
...,...,...,...,...
3464,CHEMBL5197739,CC[C@H](C)[C@@H]([C@@H](CC(=O)OC(C)(C)C)OC)N(C...,-0.907421,<deepchem.feat.mol_graphs.ConvMol object at 0x...
6367,CHEMBL290106,Oc1c(Cl)cc(Cl)cc1Sc1cc(Cl)cc(Cl)c1O,-0.925634,<deepchem.feat.mol_graphs.ConvMol object at 0x...
5111,CHEMBL2315697,CC1=CCC[C@](C)(CC/C(C)=C/CC2=C(O)C(=O)C=C(N[C@...,-1.139961,<deepchem.feat.mol_graphs.ConvMol object at 0x...
5126,CHEMBL2315699,CC1=CCC[C@](C)(CC/C(C)=C/CC2=C(O)C(=O)C=C(N[C@...,-1.139961,<deepchem.feat.mol_graphs.ConvMol object at 0x...


CHEMBL4471904


# 4. Propose an assay

In [ ]:
#!pip install seaborn

In [ ]:
import seaborn
from matplotlib.pyplot import hist
from sklearn.neighbors import KernelDensity
from scipy import stats
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from scipy.cluster.hierarchy import dendrogram
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd

In [ ]:
"""
insert code here that
1. retrieves the terms for all the assays that are relevant to the target the user picked.
2. clusters the assays according to their descriptive terms
3. plots the clusters (set n_clusters = 10)
4. prints out the title of one assay from each cluster.
"""

config = dotenv_values("database_URL.env")
url = config['DATABASE_URL']
engine = create_engine(url, echo=False)

with engine.begin() as conn:
    query = text("select * from target_to_compounds;")
    target_list = pd.read_sql(query, conn)
display(target_list)


three_col_list = target_list[['assay_id', 'assay_description', 'abstract']].copy()

unique_list = []
for index, row in three_col_list.iterrows():
    if (row["assay_id"], row["assay_description"], row["abstract"]) in unique_list:
        continue
    else:
        unique_list.append((row["assay_id"], row["assay_description"], row["abstract"]))

Assay_Descriptions = [abstract for (assay_ids, assay_name, abstract) in unique_list]
Assay_Descriptions_Joined = ':: '.join(Assay_Descriptions)

print(len(unique_list))
#print(unique_list)
#print(Assay_Descriptions)


tagger = Classifier.load('hunflair')
print("Finished setting tagger")

Assay_Descriptions_List = []

sentence = Sentence(Assay_Descriptions_Joined)
tagger.predict(sentence)

Assay_Key_Words = ""
for label in sentence.get_labels():
    Assay_Key_Words += label.data_point.text
    Assay_Key_Words += " "
    #print(label.data_point.text)
    #print(Assay_Key_Words)

#adds the Descriptions of the Assays to the Assay Description List
Assay_Descriptions_List.append(Assay_Key_Words)

def tokenize_with_flair(description):
    sentence = Sentence(description)
    tagger.predict(sentence)
    return [ l.data_point.text for l in sentence.get_labels() ]



Assay_Count_Vect = CountVectorizer(tokenizer=tokenize_with_flair)
Assay_Train_Counts = Assay_Count_Vect.fit_transform(Assay_Descriptions_List)

n_clusters = 10  # number of regions
X = Assay_Train_Counts.toarray()

ward = AgglomerativeClustering(
    n_clusters = n_clusters, linkage="ward", connectivity=None, compute_full_tree= True,compute_distances= True
)
ward.fit(X)

print(ward.distances_)
print(ward.distances_.shape)

X_Hist = ward.distances_
#hist(X_Hist)
kernel = stats.gaussian_kde(X_Hist)
print(kernel(X_Hist))

#first dendogram
seaborn.clustermap(X,method='ward')

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(Assay_Train_Counts)


# print idf values
df_idf = pd.DataFrame(tfidf_transformer.idf_,index=Assay_Count_Vect.get_feature_names(),columns=["idf_weights"])

# sort ascending
df_idf.sort_values(by=['idf_weights'])

# count matrix
count_vector=Assay_Count_Vect.transform(Assay_Descriptions_List)

# tf-idf scores
tf_idf_vector=tfidf_transformer.transform(count_vector)
print(tf_idf_vector)
tf_idf_vector.shape

feature_names = Assay_Count_Vect.get_feature_names()

#get tfidf vector for first document
first_document_vector=tf_idf_vector[0]

#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)


#second dendogram
seaborn.clustermap(tf_idf_vector.toarray(),method='ward')

Finished Installing flair
200
done
ENSG00000184292
0
0
ENSG00000198900
456
456
ENSG00000120217
34
490


,target_ENSG,compound_id,smiles,type,standard_value,assay_id,assay_type,assay_description,document_ID,abstract
0,ENSG00000198900,CHEMBL3968023,C[n+]1ccc2c3c(nc4cc5c(cc42)OCO5)-c2ccccc2-c31....,IC50,0.45,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
1,ENSG00000198900,CHEMBL3914791,C[n+]1c2c3c(nc(-c4ccccc4)cc3c3ccccc31)-c1ccccc...,IC50,0.475,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
2,ENSG00000198900,CHEMBL3896867,[Br-].c1ccc(C[n+]2ccc3c4c(nc5cc6c(cc53)OCO6)-c...,IC50,0.52,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
3,ENSG00000198900,CHEMBL3889816,[Br-].c1ccc(C[n+]2ccc3c4c(nc5ccccc53)-c3ccccc3...,IC50,0.57,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
4,ENSG00000198900,CHEMBL3984603,Cc1cc2c3c(nc4ccccc42)-c2ccccc2-c3[n+]1C.[I-],IC50,0.7,CHEMBL3867357,B,Inhibition of human DNA topoisomerase 1 using ...,CHEMBL3865847,"Eupolauridine, an indenonaphthyridine alkaloid..."
...,...,...,...,...,...,...,...,...,...,...
485,ENSG00000120217,CHEMBL4791550,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CN)C(=O)N[...,IC50,5600.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...
486,ENSG00000120217,CHEMBL4798338,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CNC(=O)[C@...,IC50,19200.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...
487,ENSG00000120217,CHEMBL366760,O=C(/C=C/c1ccc(O)c(O)c1)O[C@H]1C[C@@](O)(C(=O)...,IC50,36560.0,CHEMBL4613347,B,Antagonist activity against PDL1 (unknown origin),CHEMBL4613205,Blockade the interaction of the programmed cel...
488,ENSG00000120217,CHEMBL4790749,CC[C@H](C)[C@H](NC(=O)[C@H](C)NC(=O)CNC(=O)[C@...,IC50,80700.0,CHEMBL4688873,B,Antagonist activity at human PDL1 assessed as ...,CHEMBL4680328,Blocking the interaction of programmed cell de...


Finished setting tagger
